# パラメータの設定

---

Guacamole環境を構築するのに必要となるパラメータを入力します。

## 全体構成

![構成](images/guacamole-001-01.png)

# VCノードに関するパラメータ

Guacamole環境を構築するホストとなるVCノードに関するパラメータを指定します。

この章では、まずVCPの操作を行うために必要となるアクセストークンの値を入力します。次に構築環境のホストとなるVCノードに関するパラメータの指定を行います。最後に入力したパラメータを他のNotebookから参照できるようにNotebook環境のファイルに保存します。

![VCノード設定](images/guacamole-001-02.png)

## VCCアクセストークンの入力

VCノードを起動するにはVC Controller(VCC)にアクセスして、操作を行う必要があります。VCCにアクセスするために必要となるアクセストークンをここで入力します。

次のセルを実行すると入力枠が表示されるのでアクセストークンの値を入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [1]:
from getpass import getpass
vcc_access_token = getpass()

········


入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [2]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
2018-09-XX XX:XX:XX,XXX - ERROR - config vc failed: http_status(403)
2018-09-XX XX:XX:XX,XXX - ERROR - 2018/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

## クラウドプロバイダの指定

VCノードを起動するプロバイダを選択します。

> ここで指定できる値は `aws` または `azure` です。

In [3]:
# (例)
# vc_provider = 'aws'
# vc_provider = 'azure'

vc_provider = 'aws'

## VCノードに割り当てるリソース量の指定


VCノードに割り当てるリソース量を`spec`オブジェクトに指定します。

`spec`オブジェクトの全てのパラメータを毎回設定するのは煩雑になるので、VCP SDKでは典型的な構成のパラメータセットを事前に定義しています。事前に定義した`spec`パラメータセットのことを`flavor`と呼んでいます。`spec`に設定できるパラメータはクラウドプロバイダ毎に異なるので `flavor`もプロバイダ毎にの定義されています。

次のセルを実行すると `vc_provider` に設定したプロバイダに対応する `flavor` の一覧が表示されます。

In [4]:
vcp.df_flavors(vc_provider)

,flavor,instance_type,volume_size,volume_type
0,small,m4.large,8,gp2
1,medium,m4.xlarge,40,gp2
2,large,m4.2xlarge,100,gp2
3,gpu,p2.xlarge,40,gp2


表示された `flavor` の値から一つを選択して、次のセルに指定してください。

In [5]:
vc_flavor = 'small'

このハンズオンでは演習環境のコンテナイメージとして、サイズが３～５GBのものを利用することを想定しています。
`small`の `flavor`に対応するルートボリュームのサイズ８GBでは、演習環境コンテナを起動するのにディスク容量が不足してしまいます。そこでルートボリュームサイズを`flavor`に定義されている値から変更します。

次のセルでルートボリュームサイズ(GB)を指定してください。

In [6]:
vc_disk_size = 16

## プライベートIPアドレスの指定

Guacamoleを実行するVCノードはサーバの役割を担うので、事前に定めたIPアドレスを割り当てたほうが構築しやすくなります。ここでは、そのIPアドレスの指定を行います。

次のセルでVCノードに割り当てる IPアドレス（プライベート）を指定してください。

> VC管理者は、設定可能なIPアドレスの範囲を「VCP Manager Web UI」から確認することができます。

In [7]:
vc_guacamole_ipaddress = '172.30.2.100'

演習環境コンテナを実行するVCノードは固定のIPアドレスを割り当てる必要性がありません。そのため、演習環境用VCノードに対してはIPアドレスの指定を行いません。IPアドレスの指定を行わずにVCノードを起動した場合はVCCが利用可能なIPアドレスから適切な値を割り当てます。

## SSH公開鍵認証の鍵ファイルの指定

起動したVCノードにsshでログインして操作を行うためには事前にSSH公開鍵認証の公開鍵を登録する必要があります。

VCノードに登録するSSHの公開鍵ファイルのパスを次のセルで指定してください。

In [8]:
import os

# os.path.expanduser() を用いて '~/' を展開します
ssh_public_key_path = os.path.expanduser('~/.ssh/id_rsa.pub')

公開鍵に対応する秘密鍵のパスを次のセルで指定してください。

In [9]:
ssh_private_key_path = os.path.expanduser('~/.ssh/id_rsa')

## UnitGroup名の指定

VCPの構成要素は以下のようになっています。

* VCノード
  - クラウドにおける計算資源(VM/BM)
  - 例えば Amazon EC2インスタンス, Microsoft Azure VM など
* Unit
  - 同質のVCノードにより構成されている要素
  - 同じUnitに属するVCノードはCPU,メモリ等の計算資源が全て同じ設定になっている
* UnitGroup
  - 複数のUnitにより構成されている要素
  - 使用目的、ライフサイクルなどに合わせて、複数のUnitをまとめて扱うための要素

![UnitGroup](images/guacamole-001-02-B.png)

このNotebookで構築するGuacamole環境では２つのUnitを作成することを想定しています。
* Guacamole用ユニット
  - GuacamoleやGuacamoleの設定値を保存するためのデータベースのコンテナを実行するVCノードが属するユニット
  - ユニットのVCノードは１ノードのみとなる
* 演習環境用ユニット
  - 演習環境コンテナを実行するVCノードが属するユニット
  - ユニットのVCノードは必要に応じてスケールする

２つのUnitをまとめて扱うための UnitGroup の名前を、次のセルで指定してください。

In [10]:
unit_group = 'Guacamole-B'

## パラメータの保存

この章で指定したパラメータの値をファイルに保存します。

後の手順でVCノードに対する操作を、構成管理ツールの[Ansible](https://www.ansible.com/)で行います。そこで、パラメータの保存形式は `Ansible` のフォーマットに従うことにします。Ansible では `group_vars/`というディレクトリに YAML フォーマットのファイルを配置すると、そのファイルに記録されている値を変数として利用することができます。このNotebookでは `group_vars/` にあるファイルを `group_vars ファイル`と呼ぶことにします。

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

> YAMLフォーマットでファイルに値を保存するために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。

In [11]:
%run scripts/group.py

update_group_vars(
    unit_group,
    unit_group=unit_group,
    vc_provider=vc_provider,
    vc_flavor=vc_flavor,
    vc_disk_size=vc_disk_size,
    vc_guacamole_ipaddress=vc_guacamole_ipaddress,
    ssh_public_key_path=ssh_public_key_path,
    ssh_private_key_path=ssh_private_key_path,
)

上のセルがエラーになる場合は、この章で設定する必要があるパラメータが、まだ設定されていない可能性があります。この章のセルで実行していないことを確認してください。実行していないものがあった場合はそのセルを実行した後、エラーになったこの節のセルを再度実行して下さい。

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

In [12]:
!cat group_vars/{unit_group}.yml

ssh_private_key_path: /home/bit_kun/.ssh/id_rsa
ssh_public_key_path: /home/bit_kun/.ssh/id_rsa.pub
unit_group: Guacamole-B
vc_disk_size: 16
vc_flavor: small
vc_guacamole_ipaddress: 172.30.2.100
vc_provider: aws


# Guacamoleに関するパラメータ

この章ではGuacamoleに関するパラメータの指定を行います。まずGuacamoleを利用する一般ユーザのリストを指定します。次に Guacamole の管理者ユーザ(`guacadmin`)のパスワードを指定します。そして Guacamole の各種設定値の保存先となるデータベースに関するパラメータの指定を行います。最後に入力したパラメータの保存を行います。

Guacamoleの管理者パスワードと、データベースに接続するためのパスワードについては秘密情報となるので、Notebook環境には保存せず、VCCが提供している HashiCorp Vault に保存します。HashiCorp Vault は秘密情報を保存するための Key Valueストアです。保持する情報は暗号化されて記録されています。

秘密情報ではない、その他のパラメータについては前の章と同様に group_vars ファイルに記録します。

![Guacamole設定](images/guacamole-001-03.png)

## Guacamoleユーザの指定

Guacamoleに登録する一般ユーザのユーザ名のリストを指定してください。

> ここで構築するGuacamole環境ではユーザの認証情報に LDAP サーバに格納されている値を利用することを想定しています。そのためユーザ名は、事前にLDAPサーバに登録されているユーザ名と一致している必要があります。

In [13]:
guacamole_users = [
    'user01',
    'user02',
    'user03',
    'user04',
    'user05',
    'user06',
    'user07',
    'user08',
    'user09',
    'user10',
]

## 管理者パスワード

Guacamoleの管理者ユーザ`guacadmin`のパスワードを次のセルで指定してください。

> 通常は秘密情報をNotebook環境に不用意に残さないようにするため、パスワードの指定はパスワード入力欄を用いることが望ましいです。このハンズオンではタイプミスによるトラブルを避けるために、あえて文字列で直接設定を行っています。実際の運用ではこのような指定方法を用いないでください。

In [14]:
# (例)
# guacamole_admin_password = getpass()

guacamole_admin_password = 'guacamole'

Guacamoleの管理者ユーザにはデフォルトのパスワードが設定されています。Guacamole環境を起動した後に、ここで設定したパスワードに変更します。

## データベースに関するパラメータ

Guacamole では設定値をデータベースに保存します。データベースに関するパラメータをここで指定します。

Guacamoleの設定値を保存するデータベースの名前を指定します。

In [15]:
guacamole_db_name = 'guacamole'

Guacamoleの設定値を保存するデータベースのユーザ名を指定します。

In [16]:
guacamole_db_user = 'guacamole'

データベースユーザのパスワードを指定します。

> 通常は秘密情報をNotebook環境に不用意に残さないようにするため、パスワードの指定はパスワード入力欄を用いることが望ましいです。このハンズオンではタイプミスによるトラブルを避けるために、あえて文字列で直接設定を行っています。実際の運用ではこのような指定方法を用いないでください。

In [17]:
# (例)
# guacamole_db_password = getpass()

guacamole_db_password = 'guacamole'

## パラメータの保存

内容によって異なる場所に保存します。パスワード以外の秘密情報ではないパラメータについては前の章と同様にJupyter Notebook環境内の group_vars ファイルに保存します。パスワード等の秘密情報については、より安全度の高いVCCのVaultサーバに保存します。

### Vaultサーバへの保存

以下に示す秘密情報をVCCのVaultサーバに保存します。
* Guacamoleの管理者パスワード
* データベースの接続パスワード

Vaultサーバ内の保存場所のパスを指定します。

> 他の秘密情報の保存場所と重複しないようにするために UnitGroup 名の値をパスに含めています。

In [18]:
guacamole_vault_path = f'cubbyhole/{unit_group}/guacamole'
print(guacamole_vault_path)

cubbyhole/Guacamole-B/guacamole


Vaultサーバに秘密情報のパラメータを保存します。

> 次のセルでは Vault の REST API を呼び出すために Python の requests ライブラリを利用しています。Vaultサーバの REST APIや requests ライブラリの簡単な利用法については vcp tutorialの[「102-秘密情報管理サーバの利用.ipynb」](../vcp-tutorial/102-秘密情報管理サーバの利用.ipynb) を参照してください。

In [19]:
import requests

# 保存するパラメータを１つの dict にまとめる
payload = {
    'admin_password': guacamole_admin_password,
    'db_password': guacamole_db_password,
}

# Vault サーバのURLを組み立てる
url = f'{vcp.vcc_info()["vault_url"]}/v1/{guacamole_vault_path}'
print(url)

# VaultサーバのアクセストークンをHTTPヘッダに設定する
custom_headers = {
    'X-Vault-Token': vcc_access_token,
}

r = requests.post(url, headers=custom_headers, json=payload)
r

http://192.168.2.1:8200/v1/cubbyhole/Guacamole-B/guacamole


<Response [204]>

Vaultサーバへの値の保存が成功すると　`<Response [204]>` と表示されます。

### group_vars ファイルへの保存

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

> YAMLフォーマットでファイルに値を保存するために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。

In [20]:
%run scripts/group.py

update_group_vars(
    unit_group,
    guacamole_users=guacamole_users,
    guacamole_db_name=guacamole_db_name,
    guacamole_db_user=guacamole_db_user,
    guacamole_vault_path=guacamole_vault_path,
)

上のセルがエラーになる場合は、この章で設定する必要があるパラメータが、まだ設定されていない可能性があります。この章のセルで実行していないことを確認してください。実行していないものがあった場合はそのセルを実行した後、エラーになったこの節のセルを再度実行して下さい。

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

> ２章で設定したパラメータも合わせて表示されます。

In [21]:
!cat group_vars/{unit_group}.yml

guacamole_db_name: guacamole
guacamole_db_user: guacamole
guacamole_users:
- user01
- user02
- user03
- user04
- user05
- user06
- user07
- user08
- user09
- user10
guacamole_vault_path: cubbyhole/Guacamole-B/guacamole
ssh_private_key_path: /home/bit_kun/.ssh/id_rsa
ssh_public_key_path: /home/bit_kun/.ssh/id_rsa.pub
unit_group: Guacamole-B
vc_disk_size: 16
vc_flavor: small
vc_guacamole_ipaddress: 172.30.2.100
vc_provider: aws


# LDAPサーバに関するパラメータ

このNotebookで構築するGuacamole環境では、認証情報について外部のLDAPサーバに記録されている情報を利用しています。認証情報などのユーザに関する情報を保持しているLDAPサーバにアクセスするために必要となるパラメータをこの章で指定します。

![LDAP設定](images/guacamole-001-04.png)

## LDAPサーバのアドレス

LDAPサーバのホスト名を指定してください。

In [22]:
ldap_hostname = 'ldap.vcp-handson.org'

LDAPサーバのIPアドレスを指定してください。

> ハンズオン環境ではLDAPサーバがDNSに登録されていないため、回避策としてホスト名とIPアドレスの両方を指定しています。後で環境構築する際に、ここで指定したホスト名、IPアドレスの値を構築環境の `/etc/hosts` に登録します。

In [23]:
ldap_ipaddress = '172.30.2.50'

## 接続方法の指定

LDAPサーバへの接続方法を指定します。

ここで指定できる値は以下の通りです。

* `ssl`
 - TLS利用あり(ldaps)
* `starttls`
 - TLS利用あり(STARTTLS)
* `none`
 - TLS利用なし

In [24]:
ldap_encryption_method = 'ssl'

## CA証明書

LDAPとのSSL/TLS接続にCA証明書の設定が必要となる場合は、次のセルでCA証明書を格納しているファイルのパスを指定してください。

> CA証明書は事前に Notebook環境内に配置しておき、そのパスを指定してください。

In [25]:
ldap_ca_path = 'template/ldap/ldap-ca.crt'

## 検索用DNの指定

LDAPサーバから情報を取得する際に用いる検索用DN とそのパスワードを指定してください。

検索用DNを指定してください。

In [26]:
ldap_search_bind_dn = 'cn=ldap,dc=vcp-handson,dc=org'

パスワードを指定してください。

> 通常は秘密情報をNotebook環境に不用意に残さないようにするため、パスワードの指定はパスワード入力欄を用いることが望ましいです。このハンズオンではタイプミスによるトラブルを避けるために、あえて文字列で直接設定を行っています。実際の運用ではこのような指定方法を用いないでください。

In [27]:
ldap_search_bind_password = 'ldap'

## ベースDNの指定

ベースDNを指定してください。

In [28]:
ldap_base_dn = 'dc=vcp-handson,dc=org'

ユーザ情報を取得する際のベースDNを指定してください。

In [29]:
ldap_user_base_dn = 'ou=People,dc=vcp-handson,dc=org'

グループ情報を取得する際のベースDNを指定してください。

In [30]:
ldap_group_base_dn = 'ou=Group,dc=vcp-handson,dc=org'

## パラメータの保存

内容によって異なる場所に保存します。パスワード以外の秘密情報ではないパラメータについては前の章と同様にJupyter Notebook環境内の group_vars ファイルに保存します。パスワード等の秘密情報については、より安全度の高いVCCのVaultサーバに保存します。

### Vaultサーバへの保存

以下に示す秘密情報をVCCのVaultサーバに保存します。
* 情報検索用のパスワード

Vaultサーバ内の保存場所のパスを指定します。

> 他の秘密情報の保存場所と重複しないようにするために UnitGroup 名の値をパスに含めています。

In [31]:
ldap_vault_path = f'cubbyhole/{unit_group}/ldap'

Vaultサーバに秘密情報のパラメータを保存します。

> 次のセルでは Vault の REST API を呼び出すために Python の requests ライブラリを利用しています。Vaultサーバの REST APIや requests ライブラリの簡単な利用法については vcp tutorialの[「102-秘密情報管理サーバの利用.ipynb」](../vcp-tutorial/102-秘密情報管理サーバの利用.ipynb) を参照してください。

In [32]:
import requests

# 保存するパラメータを１つの dict にまとめる
payload = {
    'search_bind_password': ldap_search_bind_password,
}

# Vault サーバのURLを組み立てる
url = f'{vcp.vcc_info()["vault_url"]}/v1/{ldap_vault_path}'
print(url)

# VaultサーバのアクセストークンをHTTPヘッダに設定する
custom_headers = {
    'X-Vault-Token': vcc_access_token,
}

r = requests.post(url, headers=custom_headers, json=payload)
r

http://192.168.2.1:8200/v1/cubbyhole/Guacamole-B/ldap


<Response [204]>

Vaultサーバへの値の保存が成功すると　`<Response [204]>` と表示されます。

### group_vars ファイルへの保存

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

> YAMLフォーマットでファイルに値を保存するために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。

In [33]:
%run scripts/group.py

update_group_vars(
    unit_group,
    ldap_hostname=ldap_hostname,
    ldap_ipaddress=ldap_ipaddress,
    ldap_encryption_method=ldap_encryption_method,
    ldap_search_bind_dn=ldap_search_bind_dn,
    ldap_base_dn=ldap_base_dn,
    ldap_user_base_dn=ldap_user_base_dn,
    ldap_group_base_dn=ldap_group_base_dn,
    ldap_vault_path=ldap_vault_path,
)

if (
    'ldap_ca_path' in vars()
    or True                   # ldap_ca_path の設定が不要な場合はこの行をコメントアウトする
   ):
    update_group_vars(
        unit_group,
        ldap_ca_path=ldap_ca_path,
    )

上のセルがエラーになる場合は、この章で設定する必要があるパラメータが、まだ設定されていない可能性があります。この章のセルで実行していないことを確認してください。実行していないものがあった場合はそのセルを実行した後、エラーになったこの節のセルを再度実行して下さい。

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

> ２章、３章で設定したパラメータも合わせて表示されます。

In [34]:
!cat group_vars/{unit_group}.yml

guacamole_db_name: guacamole
guacamole_db_user: guacamole
guacamole_users:
- user01
- user02
- user03
- user04
- user05
- user06
- user07
- user08
- user09
- user10
guacamole_vault_path: cubbyhole/Guacamole-B/guacamole
ldap_base_dn: dc=vcp-handson,dc=org
ldap_ca_path: template/ldap/ldap-ca.crt
ldap_encryption_method: ssl
ldap_group_base_dn: ou=Group,dc=vcp-handson,dc=org
ldap_hostname: ldap.vcp-handson.org
ldap_ipaddress: 172.30.2.50
ldap_search_bind_dn: cn=ldap,dc=vcp-handson,dc=org
ldap_user_base_dn: ou=People,dc=vcp-handson,dc=org
ldap_vault_path: cubbyhole/Guacamole-B/ldap
ssh_private_key_path: /home/bit_kun/.ssh/id_rsa
ssh_public_key_path: /home/bit_kun/.ssh/id_rsa.pub
unit_group: Guacamole-B
vc_disk_size: 16
vc_flavor: small
vc_guacamole_ipaddress: 172.30.2.100
vc_provider: aws


# 演習環境コンテナに関するパラメータ

この章では演習環境コンテナに関するパラメータの指定を行います。まず演習環境のコンテナイメージを指定します。次に各演習環境コンテナに割り当てるリソースの上限値を指定します。そして演習環境コンテナを実行時間を指定します。

指定されたパラメータはこれまでの章と同様に group_vars ファイルに記録します。

![演習環境設定](images/guacamole-001-05.png)

## コンテナイメージ名の指定

演習環境として使用するコンテナイメージを指定してください。

In [35]:
# VCCのプライベートレジストリのアドレスを取得する
registry = vcp.vcc_info()['docker_registry']['application']

user_container_image = f'{registry}/vcp/xrdp:rstudio'
print(user_container_image)

192.168.2.1:5001/vcp/xrdp:rstudio


## コンテナイメージのカスタマイズ

演習環境として利用するコンテナイメージにパッケージを追加するなどの変更を加えることが出来ます。

> ここでは RPM パッケージを追加するだけの単純な変更のみを想定しています。コンテナイメージを大幅に変更する必要がある場合は目的に応じたコンテナイメージを別途用意してください。

ここでは、演習環境コンテナにパッケージを追加する例として R と [RStudio](https://www.rstudio.com/) を追加してみます。

変更元となるコンテナイメージ名を指定してください。

> 元となるコンテナイメージではSSH または RDP で外部から接続出来ることを想定しています。

In [36]:
user_container_image_base = vcp.vcc_info()['docker_registry']['base'] + '/vcp/xrdp:devel'
print(user_container_image_base)

192.168.2.1:5000/vcp/xrdp:devel


追加するパッケージのリストを指定してください。

In [37]:
user_container_additional_packages = [
    'R',
    'git',
    'https://download1.rstudio.org/rstudio-1.1.463-x86_64.rpm',
]

## コンテナに割り当てるリソース量

演習環境コンテナに割り当てるリソースを指定します。

演習環境用として利用するVCノードでは複数の演習環境コンテナを起動します。その際に１つのコンテナに割り当てるリソース量をここで指定します。

構築環境用VCノードでは実行中のコンテナに割り当てているリソースの総量が、そのVCノードのリソース量を超えないようコンテナの起動を行います。そのため、ここで指定したコンテナのリソース量と１ノードあたりのVCノードのリソース量の関係から、VCノードで起動できるコンテナ数が定まります。起動中のVCノードに割り当て可能なリソースがない場合は、割り当て可能なリソースができるまで新たなコンテナの起動ができなくなります。

AWS EC2の m4.large インスタンスでは１ノードあたりのリソース量は以下のようになっています。

* CPU(MHz): 4600
* メモリ(MB): 8000

コンテナに割り当てるCPUのリソース量(MHz)を指定してください。

In [38]:
user_container_resource_cpu = 850

コンテナに割り当てるメモリ量(MB)を指定してください。

In [39]:
user_container_resource_memory = 1500

## コンテナの実行時間の指定

新たなコンテナにリソースを割り当てることができるようにするために、演習環境コンテナは起動後一定時間が経過した後に自動的に停止ようになっています。コンテナを停止させるまでの時間を指定してください。指定する時間の単位は分になっています。

In [40]:
user_container_timeout = 60

## パラメータの保存

ここまで指定を行ったパラメータの値をファイルに保存します。

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

> YAMLフォーマットでファイルに値を保存するために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。

In [41]:
%run scripts/group.py

update_group_vars(
    unit_group,
    user_container_image=user_container_image,
    user_container_resource_cpu=user_container_resource_cpu,
    user_container_resource_memory=user_container_resource_memory,
    user_container_timeout=user_container_timeout,
)

if 'user_container_additional_packages' in vars():
    update_group_vars(
        unit_group,
        user_container_image_base=user_container_image_base,
        user_container_additional_packages=user_container_additional_packages,
    )

上のセルがエラーになる場合は、この章で設定する必要があるパラメータが、まだ設定されていない可能性があります。この章のセルで実行していないことを確認してください。実行していないものがあった場合はそのセルを実行した後、エラーになったこの節のセルを再度実行して下さい。

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

> ４章までに設定したパラメータも合わせて表示されます。

In [42]:
!cat group_vars/{unit_group}.yml

guacamole_db_name: guacamole
guacamole_db_user: guacamole
guacamole_users:
- user01
- user02
- user03
- user04
- user05
- user06
- user07
- user08
- user09
- user10
guacamole_vault_path: cubbyhole/Guacamole-B/guacamole
ldap_base_dn: dc=vcp-handson,dc=org
ldap_ca_path: template/ldap/ldap-ca.crt
ldap_encryption_method: ssl
ldap_group_base_dn: ou=Group,dc=vcp-handson,dc=org
ldap_hostname: ldap.vcp-handson.org
ldap_ipaddress: 172.30.2.50
ldap_search_bind_dn: cn=ldap,dc=vcp-handson,dc=org
ldap_user_base_dn: ou=People,dc=vcp-handson,dc=org
ldap_vault_path: cubbyhole/Guacamole-B/ldap
ssh_private_key_path: /home/bit_kun/.ssh/id_rsa
ssh_public_key_path: /home/bit_kun/.ssh/id_rsa.pub
unit_group: Guacamole-B
user_container_additional_packages:
- R
- git
- https://download1.rstudio.org/rstudio-1.1.463-x86_64.rpm
user_container_image: 192.168.2.1:5001/vcp/xrdp:rstudio
user_container_image_base: 192.168.2.1:5000/vcp/xrdp:devel
user_container_resource_cpu: 850
user_container_resource_memory: 1500
use

# NFSに関するパラメータ

ここで構築する環境では、演習環境のホームディレクトリとしてNFSサーバを利用することを想定しています。この章ではNFSサーバに関するパラメータを指定します。

![NFS設定](images/guacamole-001-06.png)

## ドメイン名

NFSのドメイン名を指定してください。

In [43]:
nfs_domain = 'vcp-handson.org'

## サーバアドレス

NFSサーバのアドレスを指定してください。

In [44]:
nfs_address = '172.30.2.50'

## サーバ側ディレクトリ

ホームディレクトリとしてマウントするサーバ側のディレクトリを指定してください。

In [45]:
nfs_directory = '/'

## マウントオプション

NFSをマウントする際のオプションを指定してください。

In [46]:
nfs_mount_options = 'nfsvers=4.2'

## パラメータの保存

ここまで指定を行ったパラメータの値をファイルに保存します。

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

> YAMLフォーマットでファイルに値を保存するために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。

In [47]:
%run scripts/group.py

update_group_vars(
    unit_group,
    nfs_domain=nfs_domain,
    nfs_address=nfs_address,
    nfs_directory=nfs_directory,
    nfs_mount_options=nfs_mount_options,
)

上のセルがエラーになる場合は、この章で設定する必要があるパラメータが、まだ設定されていない可能性があります。この章のセルで実行していないことを確認してください。実行していないものがあった場合はそのセルを実行した後、エラーになったこの節のセルを再度実行して下さい。

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

> ５章までに設定したパラメータも合わせて表示されます。

In [48]:
!cat group_vars/{unit_group}.yml

guacamole_db_name: guacamole
guacamole_db_user: guacamole
guacamole_users:
- user01
- user02
- user03
- user04
- user05
- user06
- user07
- user08
- user09
- user10
guacamole_vault_path: cubbyhole/Guacamole-B/guacamole
ldap_base_dn: dc=vcp-handson,dc=org
ldap_ca_path: template/ldap/ldap-ca.crt
ldap_encryption_method: ssl
ldap_group_base_dn: ou=Group,dc=vcp-handson,dc=org
ldap_hostname: ldap.vcp-handson.org
ldap_ipaddress: 172.30.2.50
ldap_search_bind_dn: cn=ldap,dc=vcp-handson,dc=org
ldap_user_base_dn: ou=People,dc=vcp-handson,dc=org
ldap_vault_path: cubbyhole/Guacamole-B/ldap
nfs_address: 172.30.2.50
nfs_directory: /
nfs_domain: vcp-handson.org
nfs_mount_options: nfsvers=4.2
ssh_private_key_path: /home/bit_kun/.ssh/id_rsa
ssh_public_key_path: /home/bit_kun/.ssh/id_rsa.pub
unit_group: Guacamole-B
user_container_additional_packages:
- R
- git
- https://download1.rstudio.org/rstudio-1.1.463-x86_64.rpm
user_container_image: 192.168.2.1:5001/vcp/xrdp:rstudio
user_container_image_base: 192.

# リバースプロキシに関するパラメータ

構築するGuacamole環境ではGuacamoleへのアクセスにSSL/TLSを利用するように設定します。SSL/TLSの設定は Guacamoleを実行しているTomcatで直接行うのではなく、その前段に配置したリバースプロキシで行うことにします。リバースプロキシには [nginx](https://nginx.org/) を用います。

nginx に関するパラメータの指定をこの章で行います。

![リバースプロキシ設定](images/guacamole-001-07.png)

## 証明書の指定

SSL/TLS のために設定するサーバ証明書と鍵ファイルを指定してください。

In [49]:
# サーバ証明書
nginx_tls_cert = 'template/nginx/certs/server.crt'

# 鍵ファイル
nginx_tls_key = 'template/nginx/certs/server.key'

## nginxの設定ファイル

nginxの設定ファイルを指定してください。

> ハンズオン環境では SSL/TLS を利用するための簡素な設定ファイルを事前に用意しています。

In [50]:
nginx_conf = 'template/nginx/default.conf'

指定された設定ファイルの内容を確認します。

In [51]:
!cat {nginx_conf}

server {
    listen 443;
    server_name {{server_name}};

    ssl on;
    ssl_certificate     /etc/nginx/certs/server.crt;
    ssl_certificate_key /etc/nginx/certs/server.key;
    ssl_protocols TLSv1.2;
    ssl_prefer_server_ciphers on;

    location / {
        proxy_pass http://guacamole:8080/guacamole/;
        proxy_buffering off;
        proxy_http_version 1.1;
        proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
        proxy_set_header Upgrade $http_upgrade;
        proxy_set_header Connection $http_connection;
        proxy_cookie_path /guacamole/ /;
        access_log off;
    }
}


> 上のセルが表示する設定ファイルの内容の３行目に `server_name` と記されている箇所があります。この行の内容は設定ファイルをVCノードに配置する際に、実際のホスト名に自動的に書き換えられます。

## パラメータの保存

ここまで指定を行ったパラメータの値をファイルに保存します。

次のセルを実行すると、この章で指定したパラメータが group_vars ファイルに保存されます。

> YAMLフォーマットでファイルに値を保存するために、事前に作成した Python のスクリプト `scripts/group.py` を利用しています。

In [52]:
%run scripts/group.py

update_group_vars(
    unit_group,
    nginx_tls_cert=nginx_tls_cert,
    nginx_tls_key=nginx_tls_key,
    nginx_conf=nginx_conf,
)

上のセルがエラーになる場合は、この章で設定する必要があるパラメータが、まだ設定されていない可能性があります。この章のセルで実行していないことを確認してください。実行していないものがあった場合はそのセルを実行した後、エラーになったこの節のセルを再度実行して下さい。

group_vars ファイルの内容を表示して保存されたパラメータを確認します。

> ６章までに設定したパラメータも合わせて表示されます。

In [53]:
!cat group_vars/{unit_group}.yml

guacamole_db_name: guacamole
guacamole_db_user: guacamole
guacamole_users:
- user01
- user02
- user03
- user04
- user05
- user06
- user07
- user08
- user09
- user10
guacamole_vault_path: cubbyhole/Guacamole-B/guacamole
ldap_base_dn: dc=vcp-handson,dc=org
ldap_ca_path: template/ldap/ldap-ca.crt
ldap_encryption_method: ssl
ldap_group_base_dn: ou=Group,dc=vcp-handson,dc=org
ldap_hostname: ldap.vcp-handson.org
ldap_ipaddress: 172.30.2.50
ldap_search_bind_dn: cn=ldap,dc=vcp-handson,dc=org
ldap_user_base_dn: ou=People,dc=vcp-handson,dc=org
ldap_vault_path: cubbyhole/Guacamole-B/ldap
nfs_address: 172.30.2.50
nfs_directory: /
nfs_domain: vcp-handson.org
nfs_mount_options: nfsvers=4.2
nginx_conf: template/nginx/default.conf
nginx_tls_cert: template/nginx/certs/server.crt
nginx_tls_key: template/nginx/certs/server.key
ssh_private_key_path: /home/bit_kun/.ssh/id_rsa
ssh_public_key_path: /home/bit_kun/.ssh/id_rsa.pub
unit_group: Guacamole-B
user_container_additional_packages:
- R
- git
- https://d

# チェック

このNotebookで設定する必要のあるパラメータが全て指定されていることを確認します。

In [54]:
%run scripts/group.py
gvars = load_group_vars(unit_group)


headings = {
    'vc_node': 'VCノードに関するパラメータ',
    'guacamole': 'Guacamoleに関するパラメータ',
    'ldap': 'LDAPサーバに関するパラメータ',
    'user': '演習環境コンテナに関するパラメータ',
    'nfs': 'NFSに関するパラメータ',
    'nginx': 'リバースプロキシに関するパラメータ',
}

required_params = {
    'vc_node': [
        'unit_group', 'vc_provider', 'vc_flavor', 'vc_guacamole_ipaddress',
        'ssh_public_key_path', 'ssh_private_key_path', 'vc_disk_size',
    ],
    'guacamole': [
        'guacamole_users', 'guacamole_db_name', 'guacamole_db_user',
        'guacamole_vault_path',
    ],
    'ldap': [
        'ldap_hostname', 'ldap_ipaddress', 'ldap_encryption_method',
        'ldap_search_bind_dn', 'ldap_base_dn', 'ldap_user_base_dn',
        'ldap_group_base_dn', 'ldap_vault_path',
    ],
    'user': [
        'user_container_image', 'user_container_timeout',
        'user_container_resource_cpu', 'user_container_resource_memory',
    ],
    'nfs': [
        'nfs_domain', 'nfs_address', 'nfs_directory',
        'nfs_mount_options',
    ],
    'nginx': [
        'nginx_tls_cert', 'nginx_tls_key', 'nginx_conf',
    ],
}

import sys

for k, v in required_params.items():
    rem = set(v) - gvars.keys()
    if rem:
        print(f"ERROR: 「{headings[k]}」に設定されていないパラメータがあります: {rem}", file=sys.stderr)
        raise RuntimeError()

上のセルがエラーになる場合は、設定する必要があるパラメータが group_vars ファイルに記録されていません。
設定項目が不足している章の名前が表示されますので、対応する章に実行していないセルがないことを確認してください。実行していないセルを全て実行した後に、エラーになったこの上のセルを再度実行して確認を行ってください。